In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Add config.json file to the workspace
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential, path="config.json")

Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/jcharley4/code/Users/jcharley/config.json


## Create Online Endpoint

In [7]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

# create an online endpoint
online_endpoint = ManagedOnlineEndpoint(
    name="taxi-online-endpoint-ju1", 
    description="Taxi online endpoint",
    auth_mode="aml_token",
)
ml_client.online_endpoints.begin_create_or_update(
    online_endpoint,   
)


ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://taxi-online-endpoint-ju1.westeurope.inference.ml.azure.com/score', 'swagger_uri': 'https://taxi-online-endpoint-ju1.westeurope.inference.ml.azure.com/swagger.json', 'name': 'taxi-online-endpoint-ju1', 'description': 'Taxi online endpoint', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/66914bb5-9cb2-4f6d-a84d-8ff900446b22/resourcegroups/learning/providers/microsoft.machinelearningservices/workspaces/test_learn/onlineendpoints/taxi-online-endpoint-ju1', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/66914bb5-9cb2-4f6d-a84d-8ff900446b22/providers/Microsoft.MachineLearningServices/locations/westeurope/mfeOperationsStatus/oe:75310cf9-c772-4f21-9638-4129bab98f70:61f94e53-337e-4c20-a4cd-1d835f3458f4?api-version=2022-02-01-preview'}, 'id': '/subscriptions/66914bb5-9cb2-4f6d-a84d-8ff900446b22/resourceGroups/Learning/providers/Micro

## 2. Create Online Deployment

To create a deployment to online endpoint, you need to specify the following elements:

* Model files (or specify a registered model in your workspace)
* Scoring script - code needed to do scoring/inferencing
* Environment - a Docker image with Conda dependencies, or a dockerfile
* Compute instance & scale settings

Note that if you're deploying **MLFlow models**, there's no need to provide **a scoring script** and execution **environment**, as both are autogenerated.

We can create an **online deployment** with cli v2 or sdk v2 using the following syntax:


In [8]:
# create online deployment
from azure.ai.ml.entities import ManagedOnlineDeployment, Model, Environment

model = "taxi-model@latest"

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name="taxi-online-endpoint-ju1",
    model=model,
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment
)


Check: endpoint taxi-online-endpoint-ju1 exists
Creating/updating online deployment blue 

.......................................................................................................................